# AB2017 - Python ile Pratik Makine Öğrenimi

Bu dökümanda İngilizce kısa mesaj (SMS) içeriklerden spam tespiti yapan bir sistem eğiteceğiz. Gerekli bazı importları yaparak başlıyoruz:

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

Veri setini tab ile ayrışmış metin dosyasından yüklüyoruz. Test yüzdesi olarak %20 seçelim.

In [2]:
from sklearn.model_selection import train_test_split

dataset = pd.read_csv("data/SMSSpamCollection", sep='\t', header=None)

X = dataset[1]
y = dataset[0]

X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2)


Önceki sınıflandırma sistemlerinden farklı olarak, metin sınıflandırmada bize yardımcı bir araç gerekiyor: **TfidfVectorizer**. Bu sınıfı kullanarak değişken karakter sayısındaki metinlerden daima aynı sayıda öznitelik çıkartacağız ve istediğimiz modelde kullanacağız.

In [6]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train_raw)
X_test = vectorizer.transform(X_test_raw)

feature_size = X_train.shape[1]

Önceki örneklerde olduğu gibi **GridSearchCV** kullanalım. **max_features** parametresine verilen değer sayısını az tutmakta fayda var zira kullandığımız **TfidfVectorizer** sistemi varsayılan haliyle bu sistemde 7000'den fazla öznitelik üretecek.

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV

parameters = {
    'max_depth': list(range(1, 21)),
    'max_features': list(range(1, feature_size + 1, 500))
}

grid = GridSearchCV(estimator=DecisionTreeClassifier(),
                    param_grid=parameters)

grid.fit(X_train, y_train)

print "Eğitim Skoru: %.4f" % grid.best_score_
print "Test Skoru: %.4f" % grid.best_estimator_.score(X_test, y_test)

Eğitim Skoru: 0.9704
Test Skoru: 0.9587


In [9]:
predictions = grid.best_estimator_.predict(X_test)

for (i, pred) in enumerate(predictions[:5]):
    print("Tahmin: %s\t Mesaj: %s\n" % (pred, X_test_raw.iloc[[i]].iat[0]))

Tahmin: ham	 Mesaj: Hmm. Shall i bring a bottle of wine to keep us amused? Just joking! I'll still bring a bottle. Red or white? See you tomorrow

Tahmin: ham	 Mesaj: But i have to. I like to have love and arrange.

Tahmin: spam	 Mesaj: Urgent Urgent! We have 800 FREE flights to Europe to give away, call B4 10th Sept & take a friend 4 FREE. Call now to claim on 09050000555. BA128NNFWFLY150ppm

Tahmin: ham	 Mesaj: I hope you arnt pissed off but id would really like to see you tomorrow. Love me xxxxxxxxxxxxxX

Tahmin: ham	 Mesaj: on a Tuesday night r u 4 real



Model üstünden kendimiz tahminler yapmak istiyorsak, sınıflandırılacak metini önce vectorizer nesnesinden geçirip, öznitelik çıkarmamız gerecek.

In [10]:
message_vec = vectorizer.transform(["Free buy sms travel won urgent call claim"])
grid.best_estimator_.predict(message_vec)[0]

'spam'

Önceki sistemlerden farklı olarak sadece modeli değil, vectorizer nesnesini de kaydetmek zorundayız.

In [11]:
import pickle

with open("data/spam_detector_vectorizer.pkl", 'wb') as f:
    pickle.dump(vectorizer, f)
    
with open("data/spam_detector.pkl", 'wb') as f:
    pickle.dump(grid.best_estimator_, f)